In [1]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pyomo.environ as pe

In [2]:
# Taken from https://jckantor.github.io/CBE30338/06.99-Pyomo-Examples.html

V = 40     # liters
kA = 0.5   # 1/min
kB = 0.1   # l/min
CAf = 2.0  # moles/liter

# create a model instance
model = pe.ConcreteModel()

# create x and y variables in the model
model.q = pe.Var(domain=pe.NonNegativeReals, initialize=0.0)

# add a model objective
model.objective = pe.Objective(expr = model.q*V*kA*CAf/(model.q + V*kB)/(model.q + V*kA), sense=pe.maximize)

# compute a solution using ipopt for nonlinear optimization
results = pe.SolverFactory('scip', executable='C:/Program Files/SCIPOptSuite 8.0.3/bin/scip.exe').solve(model)
model.pprint()


# print solutions
qmax = model.q()
CBmax = model.objective()
print('\nFlowrate at maximum CB = ', qmax, 'liters per minute.')
print('\nMaximum CB =', CBmax, 'moles per liter.')
print('\nProductivity = ', qmax*CBmax, 'moles per minute.')

1 Var Declarations
    q : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 8.944095772551782 :  None : False : False : NonNegativeReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40.0*q/(q + 4.0)/(q + 20.0)

2 Declarations: q objective

Flowrate at maximum CB =  8.944095772551782 liters per minute.

Maximum CB = 0.9549150280461883 moles per liter.

Productivity =  8.540851465494079 moles per minute.


In [3]:
# Same thing but with function definitions

# Create a model instance
model = pe.ConcreteModel()

# Create model parameters -> parameters are not variables, they are constants
model.V = pe.Param(initialize=V)
model.kA = pe.Param(initialize=kA)
model.kB = pe.Param(initialize=kB)
model.CAf = pe.Param(initialize=CAf)

# Variables are the things we want to solve for
model.q = pe.Var(domain=pe.NonNegativeReals, initialize=0.0)

# Create a constraint with a function -> constraints are things that hold true and cannot be violated
def _constraint_one(m):
    return m.q <= 10

# Assign the constraint to the model
model.constraint_one = pe.Constraint(rule=_constraint_one)

# Add a model objective -> objective is the thing we want to maximize or minimize
def _objective_function(m):
    return m.q * m.V * m.kA * m.CAf / (m.q + m.V * m.kB) / (m.q + m.V * m.kA)
model.objective = pe.Objective(expr=_objective_function, sense=pe.maximize)

# Create a solver instance -> Change this according to the problem. SCIP is a good general purpose solver
results = pe.SolverFactory('scip', executable='C:/Program Files/SCIPOptSuite 8.0.3/bin/scip.exe').solve(model)
model.pprint()


# print solutions
qmax = model.q()
CBmax = model.objective()
print('\nFlowrate at maximum CB = ', qmax, 'liters per minute.')
print('\nMaximum CB =', CBmax, 'moles per liter.')
print('\nProductivity = ', qmax*CBmax, 'moles per minute.')

4 Param Declarations
    CAf : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :   2.0
    V : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :    40
    kA : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :   0.5
    kB : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :   0.1

1 Var Declarations
    q : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 8.944540051188161 :  None : False : False : NonNegativeReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40.0*q/(q + 4.0)/(q + 20.0)

1 Constraint Declarations
    constraint_one : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :